In [1]:
%cd ~/ipython-notebook/dqn4wirelesscontrol/
import sys
import time
import pandas as pd
pd.set_option('mode.chained_assignment', None)  # block warnings due to DataFrame value assignment
# Project modules
from sleep_control.integration import Emulation
import sleep_control.traffic_emulator
import sleep_control.traffic_server
from sleep_control.controller import QController, DummyController
import qlearning.qtable
import qlearning.qnn
from qlearning.qtable import QAgent
from qlearning.qnn import QAgentNN
from qlearning.mixin import PhiMixin

Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, CuDNN 4007)
/usr/local/lib/python2.7/dist-packages/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


/home/admin-326/Data/ipython-notebook/dqn4wirelesscontrol


# Setting-up data

In [2]:
std_out = sys.stdout

In [3]:
fake_df = pd.DataFrame()
head_dt = pd.to_datetime("2014-09-25 09:20:31.474000")
tail_dt = pd.to_datetime("2015-01-31 15:36:36.279000") 

In [4]:
class QAgentNNPhi(PhiMixin, QAgentNN):
    def __init__(self, **kwargs):
        super(QAgentNNPhi, self).__init__(**kwargs)

# Exp1 mu1 phi1

In [18]:
te = sleep_control.traffic_emulator.PoissonEmulator(session_df=fake_df, 
                                                    head_datetime=head_dt, tail_datetime=tail_dt, time_step=pd.Timedelta(seconds=2), 
                                                    mu=1,
                                                    verbose=2)
ts = sleep_control.traffic_server.TrafficServer(cost=5, verbose=2)
actions = [(True, None), (False, 'serve_all')]
phi_length = 1
range_state_slice = [(0, 10), (0, 10), (0, 10), (0, 1), (0, 1)]
agent = QAgentNNPhi(
        phi_length=phi_length,
        dim_state=(1, phi_length, 3+2),
        range_state=[[range_state_slice]*phi_length],
        actions=actions,
        learning_rate=0.01, reward_scaling=10, reward_scaling_update='fixed', batch_size=100,
        freeze_period=50, memory_size=200, num_buffer=2,
        alpha=0.5, gamma=0.5, explore_strategy='epsilon', epsilon=0.02,
        verbose=2)
c = QController(agent=agent)
emu = sleep_control.integration.Emulation(te=te, ts=ts, c=c)

New TrafficEmulator with parameters:
  head=2014-09-25 09:20:31.474000
  tail=2015-01-31 15:36:36.279000
  time_step=0 days 00:00:02
  epoch=0
  verbose=2


In [ ]:
# %%prun -T prun_log_9.txt -D prun_log_9.stat  # for code profiling
t = time.time()
log_file = open("sleep_control/experiments/log/message_2016-6-4_1656.log","w")
sys.stdout = log_file
TOTAL_EPOCHS = 3600*24*7/2  # 7 days
while emu.epoch is not None and emu.epoch<TOTAL_EPOCHS:
    # log time
    print "Epoch {}, ".format(emu.epoch),
    left = emu.te.head_datetime + emu.te.epoch*emu.te.time_step
    right = left + emu.te.time_step
    print "{} - {}".format(left.strftime("%Y-%m-%d %H:%M:%S"), right.strftime("%Y-%m-%d %H:%M:%S"))
    emu.step()
    print
sys.stdout = std_out
time.time()-t

In [17]:
sys.stdout = std_out